In [6]:
from ohlcv_data import OHLCVDataLoader
from momentumStrat import MomentumStratOHLCV
from MAstrat import MAStrat
from RSIStrat import RSIStrat
from ohlcv_backtester import OHLCVBacktester
from ohlcv_analysis import OHLCVAnalysis
import os

In [7]:
# 1. Load and clean data
folder_path = "/Users/elyazidbenkhadra/Downloads/stock_market_data/sp500/csv"
data = OHLCVDataLoader.load_all_data(folder_path, clean=True, years=20)

2025-04-21 13:20:44,488 - INFO - Loaded and combined data from 403 files. Final shape: (4442, 2418)


In [12]:
# 2. Instantiate strategy
movingaverage = MAStrat(name="Moving Average", short_window=50, long_window=200)
RSI = RSIStrat(name="RSI")
hybrid = MomentumStratOHLCV(name="Momentum Hybrid", short_window =50, long_window=200)

In [13]:
# 3. Run backtest (multi-asset)
backtester_ma = OHLCVBacktester(data, movingaverage, fee_perc=0.0005)
backtester_rsi = OHLCVBacktester(data, RSI, fee_perc=0.0005)
backtester_hybrid = OHLCVBacktester(data, hybrid, fee_perc=0.0005)
asset_states1, portfolio_equity_curve1 = backtester_ma.run()
asset_states2, portfolio_equity_curve2 = backtester_rsi.run()
asset_states3, portfolio_equity_curve3 = backtester_hybrid.run()


=== Running backtest for strategy: Moving Average ===


2025-04-21 13:22:03,133 - INFO - Backtest complete. Trades: [27, 14, 26, 16, 17, 21, 25, 24, 26, 26, 25, 25, 31, 30, 28, 30, 21, 28, 27, 24, 26, 14, 8, 24, 22, 21, 43, 23, 22, 15, 31, 22, 25, 27, 23, 28, 26, 22, 29, 22, 28, 23, 29, 24, 30, 30, 32, 32, 31, 5, 31, 22, 26, 26, 29, 39, 26, 10, 33, 27, 28, 22, 29, 27, 25, 23, 30, 24, 24, 23, 18, 24, 32, 14, 26, 36, 32, 28, 17, 19, 34, 22, 12, 18, 25, 26, 20, 33, 23, 10, 25, 26, 24, 27, 8, 24, 20, 36, 30, 29, 20, 19, 27, 31, 24, 18, 22, 22, 30, 33, 18, 32, 23, 24, 26, 30, 20, 32, 24, 24, 25, 29, 25, 20, 20, 34, 28, 22, 20, 28, 15, 28, 14, 34, 20, 22, 26, 22, 19, 28, 26, 16, 12, 23, 21, 4, 18, 6, 22, 27, 17, 27, 37, 27, 33, 24, 25, 26, 14, 18, 29, 23, 26, 33, 18, 19, 17, 25, 15, 8, 34, 29, 14, 16, 35, 20, 26, 26, 23, 27, 25, 20, 28, 27, 30, 24, 0, 18, 24, 28, 9, 40, 33, 21, 23, 20, 26, 21, 18, 28, 22, 25, 31, 5, 20, 7, 4, 18, 22, 36, 24, 22, 36, 32, 20, 37, 34, 28, 25, 31, 16, 24, 33, 24, 28, 29, 18, 20, 20, 18, 27, 25, 25, 28, 30, 24, 31, 20


=== Running backtest for strategy: RSI ===


2025-04-21 13:22:12,065 - INFO - Backtest complete. Trades: [96, 112, 99, 108, 62, 90, 104, 102, 112, 112, 108, 104, 100, 106, 96, 111, 102, 120, 117, 95, 111, 58, 36, 116, 108, 108, 121, 90, 108, 54, 106, 109, 113, 102, 100, 110, 110, 107, 105, 88, 107, 106, 102, 103, 111, 104, 116, 92, 109, 5, 108, 110, 114, 96, 102, 116, 86, 143, 96, 98, 99, 103, 128, 100, 112, 102, 122, 104, 108, 113, 54, 100, 119, 70, 88, 120, 102, 105, 120, 121, 108, 116, 112, 108, 113, 101, 60, 101, 104, 54, 118, 117, 116, 94, 39, 105, 118, 107, 98, 102, 90, 83, 112, 100, 95, 105, 110, 94, 99, 120, 103, 107, 110, 105, 99, 114, 102, 108, 113, 100, 114, 100, 93, 106, 87, 110, 97, 111, 111, 113, 63, 122, 65, 98, 114, 106, 110, 102, 116, 107, 106, 72, 128, 96, 90, 25, 76, 26, 108, 94, 110, 97, 116, 106, 108, 84, 104, 111, 104, 68, 106, 111, 103, 95, 85, 90, 108, 105, 65, 62, 98, 100, 50, 100, 114, 100, 96, 116, 106, 101, 117, 98, 102, 108, 102, 101, 9, 107, 102, 106, 36, 120, 100, 100, 110, 88, 106, 106, 95, 116, 12


=== Running backtest for strategy: Momentum Hybrid ===


2025-04-21 13:22:21,409 - INFO - Backtest complete. Trades: [28, 16, 24, 16, 16, 18, 26, 26, 26, 22, 23, 27, 26, 30, 28, 27, 21, 30, 27, 26, 24, 12, 8, 24, 18, 20, 37, 20, 22, 14, 28, 20, 25, 24, 24, 26, 22, 20, 27, 18, 26, 23, 26, 24, 30, 30, 28, 22, 27, 3, 26, 24, 26, 26, 24, 32, 26, 32, 30, 26, 30, 20, 27, 29, 26, 20, 30, 24, 26, 23, 18, 24, 31, 14, 24, 28, 28, 28, 17, 21, 32, 22, 26, 20, 25, 28, 18, 27, 25, 12, 26, 26, 24, 23, 7, 24, 20, 31, 28, 26, 20, 15, 28, 30, 26, 18, 20, 22, 29, 26, 19, 24, 20, 24, 26, 28, 20, 32, 22, 24, 22, 24, 27, 22, 20, 28, 26, 18, 19, 26, 15, 26, 14, 30, 18, 22, 26, 20, 18, 24, 26, 16, 16, 18, 19, 4, 16, 10, 24, 27, 19, 29, 34, 24, 29, 22, 23, 28, 14, 16, 31, 25, 28, 27, 18, 16, 18, 25, 15, 8, 32, 28, 14, 16, 35, 22, 26, 24, 25, 29, 25, 22, 28, 27, 28, 24, 5, 20, 22, 28, 8, 36, 32, 23, 22, 22, 26, 20, 19, 28, 22, 22, 33, 3, 20, 7, 8, 14, 20, 30, 24, 22, 28, 32, 18, 28, 32, 24, 25, 28, 18, 24, 28, 24, 26, 24, 18, 18, 20, 18, 25, 26, 25, 28, 28, 24, 27, 2

In [16]:
# 4. Analyze and visualize (multi-asset, portfolio focus)
analysis1 = OHLCVAnalysis(asset_states1, portfolio_equity_curve1, data, strategy= movingaverage)
analysis2 = OHLCVAnalysis(asset_states2, portfolio_equity_curve2, data,strategy= RSI)
analysis3 = OHLCVAnalysis(asset_states3, portfolio_equity_curve3, data, strategy= hybrid)
analysis1.print_portfolio_metrics(full=True)
analysis2.print_portfolio_metrics(full=True)
analysis3.print_portfolio_metrics(full=True)

# Plot portfolio PnL
OHLCVAnalysis.plot_multiple_portfolio_pnl(
    [
        (portfolio_equity_curve1, movingaverage),
        (portfolio_equity_curve2, RSI),
        (portfolio_equity_curve3, hybrid)
    ],
    data
)

# Plot portfolio cash and equity allocation
analysis1.plot_portfolio_allocations()
analysis1.plot_average_position_size()
analysis1.plot_portfolio_allocations_pct()
analysis1.plot_pnl_decomposition()

# (Optional) For a specific asset, you can still use:
loaded_ticker = 'AAPL'
analysis2.plot_asset_trades(loaded_ticker)
analysis2.plot_asset_pnl_with_trades(loaded_ticker)
analysis2.plot_asset_equity_allocation_pct(loaded_ticker)
analysis2.plot_asset_allocations(loaded_ticker)


Portfolio metrics for strategy: Moving Average
  Number of assets: 403
  Average Asset PnL: 4712.97
  Variance of Asset PnL: 71950221.01
  Sharpe Ratio: 0.62
  Max Drawdown: -26.24%
  Total Return: 189.93%

Portfolio metrics for strategy: RSI
  Number of assets: 403
  Average Asset PnL: 4853.66
  Variance of Asset PnL: 41329005.07
  Sharpe Ratio: 0.55
  Max Drawdown: -33.33%
  Total Return: 195.60%

Portfolio metrics for strategy: Momentum Hybrid
  Number of assets: 403
  Average Asset PnL: 5416.40
  Variance of Asset PnL: 75536916.67
  Sharpe Ratio: 0.61
  Max Drawdown: -28.87%
  Total Return: 218.28%


AttributeError: type object 'OHLCVAnalysis' has no attribute 'plot_multiple_portfolio_pnl'